In [13]:
# from https://huggingface.co/bert-base-cased:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-cased')
out = unmasker("The person was sent to prison. His skin color was [MASK].")
# for i, token in enumerate(out):
#     if token['token_str'] in ['white', 'black']:
#         print(f"token:{token['token_str']}: {token['score']}")
out

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10653549432754517,
  'token': 1472,
  'token_str': 'different',
  'sequence': 'The person was sent to prison. His skin color was different.'},
 {'score': 0.04661107063293457,
  'token': 3655,
  'token_str': 'unknown',
  'sequence': 'The person was sent to prison. His skin color was unknown.'},
 {'score': 0.033750057220458984,
  'token': 3264,
  'token_str': 'perfect',
  'sequence': 'The person was sent to prison. His skin color was perfect.'},
 {'score': 0.03187955543398857,
  'token': 1602,
  'token_str': 'black',
  'sequence': 'The person was sent to prison. His skin color was black.'},
 {'score': 0.0290673840790987,
  'token': 1894,
  'token_str': 'red',
  'sequence': 'The person was sent to prison. His skin color was red.'}]

In [20]:
# adapted from https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
# model.to('cuda')  # if you have gpu

def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

        
predict_masked_sent("The man went to prison. His skin color was [MASK].", top_k=10)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[MASK]: 'dark'  | weights: 0.07368823140859604
[MASK]: 'gray'  | weights: 0.07224850356578827
[MASK]: 'different'  | weights: 0.06697370857000351
[MASK]: 'brown'  | weights: 0.05147457495331764
[MASK]: 'black'  | weights: 0.04427278786897659
[MASK]: 'white'  | weights: 0.03915740177035332
[MASK]: 'gone'  | weights: 0.03350396454334259
[MASK]: 'pale'  | weights: 0.029876839369535446
[MASK]: 'bad'  | weights: 0.022695040330290794
[MASK]: 'blue'  | weights: 0.018285172060132027
